In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from math import sqrt
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')


In [2]:
df=pd.read_excel('crime_data_clean.xlsx')

In [3]:
df=df.sample(n=15000)

In [4]:
df.head()

,OFFENSE_CODE_GROUP,DISTRICT,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,Lat,Long
4513,Aggravated Assault,D4,2018,9,Monday,21,Part One,NaN,NaN
9359,Motor Vehicle Accident Response,B2,2018,8,Tuesday,12,Part Three,42.339623,-71.106497
131144,Auto Theft,B3,2017,6,Thursday,15,Part One,42.290425,-71.090491
41458,Fraud,B3,2018,5,Friday,1,Part Two,42.281625,-71.083192
258315,Motor Vehicle Accident Response,B2,2016,3,Sunday,19,Part Three,42.330206,-71.083178


In [5]:
df_dummy = pd.get_dummies(df,columns=['OFFENSE_CODE_GROUP','DISTRICT',
                                      'DAY_OF_WEEK',],drop_first=True)

In [6]:
df_dummy.head()

,YEAR,MONTH,HOUR,UCR_PART,Lat,Long,OFFENSE_CODE_GROUP_Aircraft,OFFENSE_CODE_GROUP_Arson,OFFENSE_CODE_GROUP_Assembly or Gathering Violations,OFFENSE_CODE_GROUP_Auto Theft,...,DISTRICT_D4,DISTRICT_E13,DISTRICT_E18,DISTRICT_E5,DAY_OF_WEEK_Monday,DAY_OF_WEEK_Saturday,DAY_OF_WEEK_Sunday,DAY_OF_WEEK_Thursday,DAY_OF_WEEK_Tuesday,DAY_OF_WEEK_Wednesday
4513,2018,9,21,Part One,NaN,NaN,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
9359,2018,8,12,Part Three,42.339623,-71.106497,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
131144,2017,6,15,Part One,42.290425,-71.090491,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
41458,2018,5,1,Part Two,42.281625,-71.083192,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
258315,2016,3,19,Part Three,42.330206,-71.083178,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [7]:
df_dummy.UCR_PART = df_dummy.UCR_PART.map({
                                
                                'Part Three':1,
                                'Part Two':2,
                                'Part One':3                                                       
                                                                
                                })

In [8]:
df_dummy.head()

,YEAR,MONTH,HOUR,UCR_PART,Lat,Long,OFFENSE_CODE_GROUP_Aircraft,OFFENSE_CODE_GROUP_Arson,OFFENSE_CODE_GROUP_Assembly or Gathering Violations,OFFENSE_CODE_GROUP_Auto Theft,...,DISTRICT_D4,DISTRICT_E13,DISTRICT_E18,DISTRICT_E5,DAY_OF_WEEK_Monday,DAY_OF_WEEK_Saturday,DAY_OF_WEEK_Sunday,DAY_OF_WEEK_Thursday,DAY_OF_WEEK_Tuesday,DAY_OF_WEEK_Wednesday
4513,2018,9,21,3.0,NaN,NaN,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
9359,2018,8,12,1.0,42.339623,-71.106497,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
131144,2017,6,15,3.0,42.290425,-71.090491,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
41458,2018,5,1,2.0,42.281625,-71.083192,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
258315,2016,3,19,1.0,42.330206,-71.083178,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [9]:
dfs=df_dummy

In [10]:
dfs.dtypes

YEAR                       int64
MONTH                      int64
HOUR                       int64
UCR_PART                 float64
Lat                      float64
                          ...   
DAY_OF_WEEK_Saturday       uint8
DAY_OF_WEEK_Sunday         uint8
DAY_OF_WEEK_Thursday       uint8
DAY_OF_WEEK_Tuesday        uint8
DAY_OF_WEEK_Wednesday      uint8
Length: 82, dtype: object

In [11]:
dfs.isnull().sum()

YEAR                       0
MONTH                      0
HOUR                       0
UCR_PART                  69
Lat                      977
                        ... 
DAY_OF_WEEK_Saturday       0
DAY_OF_WEEK_Sunday         0
DAY_OF_WEEK_Thursday       0
DAY_OF_WEEK_Tuesday        0
DAY_OF_WEEK_Wednesday      0
Length: 82, dtype: int64

In [12]:
dfs.dropna(inplace=True)

In [13]:
dfs.isnull().sum()

YEAR                     0
MONTH                    0
HOUR                     0
UCR_PART                 0
Lat                      0
                        ..
DAY_OF_WEEK_Saturday     0
DAY_OF_WEEK_Sunday       0
DAY_OF_WEEK_Thursday     0
DAY_OF_WEEK_Tuesday      0
DAY_OF_WEEK_Wednesday    0
Length: 82, dtype: int64

In [14]:
dfs.dtypes

YEAR                       int64
MONTH                      int64
HOUR                       int64
UCR_PART                 float64
Lat                      float64
                          ...   
DAY_OF_WEEK_Saturday       uint8
DAY_OF_WEEK_Sunday         uint8
DAY_OF_WEEK_Thursday       uint8
DAY_OF_WEEK_Tuesday        uint8
DAY_OF_WEEK_Wednesday      uint8
Length: 82, dtype: object

In [15]:
cols = dfs.columns[dfs.dtypes.eq('object')]
dfs[cols] = dfs[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [16]:
X=dfs.drop('UCR_PART',axis=1)
y=dfs['UCR_PART']

# LOGISTIC REGRESSION

In [36]:
# Splitting the dataset into the Training set and Test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)


In [37]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [38]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
#Hyperparameter tuning -GridsearchCV
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression()
#solver_options = ['newton-cg', 'lbfgs', 'liblinear', 'sag']
#solver_options = [ 'liblinear', 'sag']
solver_options = [ 'liblinear']
#penalty_options=['l1', 'l2', 'elasticnet', 'none']
penalty_options=['l1', 'l2']
#multi_class_options = ['ovr', 'multinomial','auto']
multi_class_options = ['ovr','auto']
#class_weight_options = ['None', 'balanced']
class_weight_options = ['balanced']
param_grid = dict(solver = solver_options, multi_class =
                  multi_class_options, class_weight = class_weight_options,penalty=penalty_options)

grid = GridSearchCV(lr, param_grid, cv=5, scoring = 
'accuracy')
grid.fit(X, y)
grid.best_score_
grid.best_params_

 

{'class_weight': 'balanced',
 'multi_class': 'ovr',
 'penalty': 'l1',
 'solver': 'liblinear'}

In [40]:
grid.best_params_

{'class_weight': 'balanced',
 'multi_class': 'ovr',
 'penalty': 'l1',
 'solver': 'liblinear'}

In [22]:
grid.best_score_

0.9958989031950405

In [41]:
#get k fold stdv
grid.cv_results_['std_test_score'][grid.best_index_]

0.0012717771445853905

In [43]:
#make prediction
clf = LogisticRegression(class_weight='balanced',multi_class='ovr',penalty='l1',solver='liblinear',random_state = 0)
clf.fit(X_train,y_train)
y_pred_clf=clf.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
accuracy_score(y_test,y_pred_clf)
confusion_matrix(y_test,y_pred_clf)
print(classification_report(y_test,y_pred_clf))

              precision    recall  f1-score   support

         1.0       1.00      0.99      1.00      1720
         2.0       0.99      1.00      0.99      1080
         3.0       1.00      1.00      1.00       697

    accuracy                           1.00      3497
   macro avg       1.00      1.00      1.00      3497
weighted avg       1.00      1.00      1.00      3497



In [42]:
clf.score(X_test,y_test)

0.996568487274807

# KNN-CLASSIFICATION MODEL

In [44]:
X=dfs.drop('UCR_PART',axis=1)
y=dfs['UCR_PART']

In [45]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

In [46]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_scaled = sc.fit_transform(X)

In [48]:
#Hyperparameter tuning using Gridsearch
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
reg=KNeighborsClassifier()
param_dict = {
                'n_neighbors': [3,5],
                'weights' : ['uniform','distance'],
                'metric' : ['euclidean', 'manhattan']
                              
            }
clf = GridSearchCV(reg,param_dict,cv=5)
clf.fit(X_scaled,y)
clf.best_params_
clf.best_score_

0.9939935645334287

In [49]:
clf.best_params_

{'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}

In [50]:
#get k fold stdv
clf.cv_results_['std_test_score'][clf.best_index_]

0.0008896289803607617

In [51]:
# Fitting K-NN to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 3, weights='uniform', metric='euclidean')
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [53]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [54]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix,classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1708   10    2]
 [   2 1078    0]
 [   4    0  693]]


In [55]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       1.00      0.99      0.99      1720
         2.0       0.99      1.00      0.99      1080
         3.0       1.00      0.99      1.00       697

    accuracy                           0.99      3497
   macro avg       0.99      1.00      0.99      3497
weighted avg       0.99      0.99      0.99      3497



# RANDOM FOREST

In [17]:
X=dfs.drop('UCR_PART',axis=1)
y=dfs['UCR_PART']

In [18]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [19]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 0)

In [20]:
#Hyperparameter tuning using Gridsearch
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)

param_dict = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

clf = GridSearchCV(rf,param_dict,cv=5,n_jobs = -1, verbose = 2)
clf.fit(X,y)
clf.best_params_
clf.best_score_

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   58.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 17.9min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 42.9min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 57.3min finished


0.9696926273554489

In [21]:
clf.best_params_


{'bootstrap': True,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 1000}

In [22]:
clf.best_score_

0.9696926273554489

In [23]:
#get k fold stdv
clf.cv_results_['std_test_score'][clf.best_index_]

0.0028526981066457993

In [24]:
#train the model
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(bootstrap=True,max_depth=80,max_features=3,min_samples_leaf=3,
                            min_samples_split=8,n_estimators=1000)
                          
rf.fit(X_train,y_train)




RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=80, max_features=3, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [25]:
rf.score(X_test,y_test)

0.9689067201604814

In [26]:
# Predicting the Test set results
y_pred = rf.predict(X_test)

In [27]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix,classification_report
cm = confusion_matrix(y_test, y_pred)

In [29]:
print(cm)

[[3380   24    0]
 [  18 2150    0]
 [ 175    0 1232]]


In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.95      0.99      0.97      3404
         2.0       0.99      0.99      0.99      2168
         3.0       1.00      0.88      0.93      1407

    accuracy                           0.97      6979
   macro avg       0.98      0.95      0.96      6979
weighted avg       0.97      0.97      0.97      6979



# CLASSIFICATION-SVM

In [30]:
X=dfs.drop('UCR_PART',axis=1)
y=dfs['UCR_PART']

In [31]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [32]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)
sc_X = StandardScaler()
X_scaled = sc_X.fit_transform(X)

In [33]:
#Hyperparameter tuning using Gridsearch
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_dict = {
                'kernel': ['linear', 'poly', 'rbf'],
                'degree' : [2,3],
                'gamma' : [0.01, 0.1,1,10],
                'C': [0.01, 0.1,1,10]                
            }
clf = GridSearchCV(SVC(),param_dict,cv=4)
clf.fit(X_scaled,y)
clf.best_params_
clf.best_score_

0.9957727305294835

In [34]:
clf.best_params_

{'C': 0.01, 'degree': 2, 'gamma': 0.01, 'kernel': 'linear'}

In [43]:
clf.best_score_

0.9957727305294835

In [44]:
#get k fold stdv
clf.cv_results_['std_test_score'][clf.best_index_]

0.0010781271533125698

In [40]:
svm = SVC(kernel='linear',gamma=0.01,C=0.01)
svm.fit(X_train_scaled,y_train)
y_pred_svm=svm.predict(X_test_scaled)

In [41]:
rf.score(X_test_scaled,y_test)

0.9664756446991404

In [38]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
accuracy_score(y_test,y_pred_svm)
print(confusion_matrix(y_test,y_pred_svm))


[[1695   11    0]
 [   2 1082    0]
 [   0    0  700]]


In [42]:
print(classification_report(y_test,y_pred_svm))

              precision    recall  f1-score   support

         1.0       1.00      0.99      1.00      1706
         2.0       0.99      1.00      0.99      1084
         3.0       1.00      1.00      1.00       700

    accuracy                           1.00      3490
   macro avg       1.00      1.00      1.00      3490
weighted avg       1.00      1.00      1.00      3490



# CLASSIFICATION-ADABOOST

In [19]:
X=dfs.drop('UCR_PART',axis=1)
y=dfs['UCR_PART']

In [20]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 0)

In [36]:
#import libs
from sklearn.model_selection import train_test_split, GridSearchCV

In [41]:
param_grid={
    'base_estimator':[DecisionTreeClassifier(max_depth=1),DecisionTreeClassifier(max_depth=5),
                                             DecisionTreeClassifier(max_depth=10)],
           'n_estimators':[10,20,30]
           }
ABC = AdaBoostClassifier()
ada = GridSearchCV(ABC, param_grid, cv=5)
ada.fit(X_train,y_train)
ada.score(X_test,y_test)



0.9904476760764186

In [42]:
ada.best_params_

{'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=None, splitter='best'),
 'n_estimators': 30}

In [43]:
ada.best_score_

0.9914444602880365

In [44]:
#get k fold stdv
ada.cv_results_['std_test_score'][ada.best_index_]

0.0036050518153600275

In [45]:
adaboost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=30,random_state=0)
adaboost.fit(X_train,y_train)
adaboost.score(X_test,y_test)

0.9903051040775591

In [46]:
# Predicting the Test set results
y_pred = adaboost.predict(X_test)

In [47]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix,classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[3360   28    3]
 [  28 2166    8]
 [   1    0 1420]]


In [48]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         1.0       0.99      0.99      0.99      3391
         2.0       0.99      0.98      0.99      2202
         3.0       0.99      1.00      1.00      1421

    accuracy                           0.99      7014
   macro avg       0.99      0.99      0.99      7014
weighted avg       0.99      0.99      0.99      7014

